In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("../credentials.env")

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [ ]:
user_msg = """### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Make a query to list the names of the departments which employed more than 10 employees in the last 3 months.
query:
"""

# user_msg = """### Postgres SQL tables, with their properties:
# #
# # Employee(id, name, department_id)
# # Department(id, name, address)
# # Salary_Payments(id, employee_id, amount, date)
# #
# ### 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리를 만들어줘.
# query: 
# """

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

In [ ]:
user_msg = """###Can you explain what does this code do? 
# Code: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Answer: 
"""

# user_msg = """###이 코드가 무엇을 수행하는지 설명해줘?
# #Code:
# SELECT d.name
# FROM Department d
# WHERE d.id IN (
#     SELECT e.department_id
#     FROM Employee e
#     WHERE e.id IN (
#         SELECT sp.employee_id
#         FROM Salary_Payments sp
#         WHERE sp.date >= current_date - interval '3 months'
#     )
#     GROUP BY e.department_id
#     HAVING COUNT(e.id) > 10
# )
# Answer:
# """
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. All answers are in korean."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)